In [1]:
%load_ext autoreload
%autoreload 2

import radon
import torch

_avail_gpus = torch.cuda.device_count()
GPU_ID = 2  # Default GPU ID
if _avail_gpus == 0:
    print("No GPUs available -- will need one for cuda")
    device = "cpu"
else:
    print(f"Available GPUs: {_avail_gpus}")
    print(f"Using GPU ID {GPU_ID} : {torch.cuda.get_device_name(GPU_ID)}")
    torch.cuda.set_device(GPU_ID)
    device = f"cuda:{GPU_ID}"

Unable to import Radon class from radon module.


ImportError: Radon unavailable. Check Torch availability

In [ ]:
a = torch.rand(3, device=GPU_ID)
b = torch.rand(3, device=GPU_ID)
c = torch.zeros_like(a, device=GPU_ID)
radon.add_arrays(a, b, c)
print("Result:", c)

Result:Hello from CUDA thread 0: 0.59 + 0.49 = 1.07
Hello from CUDA thread 1: 0.03 + 0.51 = 0.55
Hello from CUDA thread 2: 0.50 + 0.54 = 1.04
 tensor([1.0719, 0.5461, 1.0422], device='cuda:2')


In [ ]:
r = radon.Radon()
print("Radon object created:", r)
print("Radon version:", r.version)
print("Radon working successfully on device")
print("RaysCfg object:", r.rays_cfg)

Radon object created: <radon.radon.Radon object at 0x75b66c04e960>
Radon version: 0.1.0
Radon working successfully on device
RaysCfg object: <radon.cuda._cuda_add.RaysCfg object at 0x75b80f505770>
